<a href="https://colab.research.google.com/github/Eshita781996/Pyspark_functions/blob/main/PysparkFunction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip -q install findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.7.0 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell'

In [4]:
import findspark
findspark.init()

In [64]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('delta_session').getOrCreate()

In [57]:
employees_delta_df = spark.createDataFrame([(2, ' Peter# ', 'Melbourne!', 55000.00),(5, '  Jessie*', '**Brisbane', 42000.00)],
                                           schema='emp_id int,emp_name string,emp_city string,emp_salary float')

In [32]:
def removejunkcharcter(employees_delta_df):
  for col in employees_delta_df.dtypes:
    employees_delta_df = employees_delta_df.withColumn(col[0], regexp_replace(col[0], '[^a-zA-Z0-9]', ''))
  return employees_delta_df

In [51]:
def renamecolumn(employees_delta_df,rename_dict):
  for i in rename_dict:
    employees_delta_df = employees_delta_df.withColumnRenamed(i,rename_dict[i])
  return employees_delta_df

In [53]:
def dropcolumn(employees_delta_df,drop_cols):
  for i in drop_cols:
    employees_delta_df = employees_delta_df.drop(i)
  return employees_delta_df

In [79]:
def trimcolumns(df):
  for colname in df.columns:
    df = df.withColumn(colname,trim(df[colname]))
  return df

In [80]:
employees_delta_df.show()
df = removejunkcharcter(employees_delta_df)
rename_dict = {'emp_id':'employee_id','emp_name':'employee_name','emp_city':'employee_city','emp_salary':'employee_salary'}
df_rename = renamecolumn(df,rename_dict)
drop_cols = ['employee_city']
df_drop = dropcolumn(df_rename,drop_cols)
df = trimcolumns(df_drop)
df.show()

+------+---------+----------+----------+
|emp_id| emp_name|  emp_city|emp_salary|
+------+---------+----------+----------+
|     2|  Peter# |Melbourne!|   55000.0|
|     5|  Jessie*|**Brisbane|   42000.0|
+------+---------+----------+----------+

+-----------+-------------+---------------+
|employee_id|employee_name|employee_salary|
+-----------+-------------+---------------+
|          2|        Peter|         550000|
|          5|       Jessie|         420000|
+-----------+-------------+---------------+

